In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [20]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(InceptionBlock, self).__init__()
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, ch1x1, kernel_size=1, bias=False),
            nn.BatchNorm2d(ch1x1, eps=0.001),
            nn.ReLU()
        )
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, ch3x3red, kernel_size=1, bias=False),
            nn.BatchNorm2d(ch3x3red, eps=0.001),
            nn.ReLU(),
            nn.Conv2d(ch3x3red, ch3x3, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(ch3x3, eps=0.001),
            nn.ReLU()
        )
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, ch5x5red, kernel_size=1, bias=False),
            nn.BatchNorm2d(ch5x5red, eps=0.001),
            nn.ReLU(),
            nn.Conv2d(ch5x5red, ch5x5, kernel_size=5, padding=2, bias=False),
            nn.BatchNorm2d(ch5x5, eps=0.001),
            nn.ReLU()
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),
            nn.Conv2d(in_channels, pool_proj, kernel_size=1, bias=False),
            nn.BatchNorm2d(pool_proj, eps=0.001),
            nn.ReLU(),
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = torch.cat([branch1, branch2, branch3, branch4], 1)
        return outputs

In [21]:
class InceptionConv1(nn.Module):
    def __init__(self, in_channels):
        super(InceptionConv1, self).__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.conv2 = nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(192)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool2(x)

        return x

In [48]:
class InceptionNet(nn.Module):
    def __init__(self, num_classes):
        super(InceptionNet, self).__init__()
        self.conv1 = InceptionConv1(3)

        # Inception 3 layer
        self.inception3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        # Inception 4 layers
        self.inception4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        # Inception 5 layer
        self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)

        # final layer
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        # first conv1
        x = self.conv1(x)

        # Inception 3 layer
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        # Inception 4 layer
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)

        # Inception 5 layer
        x = self.inception5a(x)
        x = self.inception5b(x)

        # final layer
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [56]:
x = torch.randn(64, 3, 224, 224)
model = InceptionNet(num_classes=1000)
x = model.conv1(x)
print(x.shape)

torch.Size([64, 192, 28, 28])


In [70]:
x = torch.randn(64, 3, 32, 32)
conv1 = nn.Conv2d(3, 64, kernel_size=1)
conv2 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

x = conv1(x)
print(x.shape)

x = conv2(x)
print(x.shape)

x = maxpool1(x)
print(x.shape)

torch.Size([64, 64, 32, 32])
torch.Size([64, 192, 32, 32])
torch.Size([64, 192, 16, 16])
